# Estimate importance factors on a Python function

Reference : https://stackoverflow.com/questions/59096111/how-to-estimate-the-taylor-decomposition-variance-from-a-python-function-with-us

## Definition of the model

In [1]:
import openturns as ot

In [2]:
def mySimulator(x):
    y0 = 1.0 - x[0] * x[1]
    y = [y0]
    return y

In [3]:
g = ot.PythonFunction(2, 1, mySimulator)

In [4]:
distX0 = ot.Normal(1.0, 3.0)
distX1 = ot.Normal(2.0, 4.0)

In [5]:
X = ot.ComposedDistribution([distX0, distX1])
X

class=ComposedDistribution name=ComposedDistribution dimension=2 copula=class=IndependentCopula name=IndependentCopula dimension=2 marginal[0]=class=Normal name=Normal dimension=1 mean=class=Point name=Unnamed dimension=1 values=[1] sigma=class=Point name=Unnamed dimension=1 values=[3] correlationMatrix=class=CorrelationMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[1] marginal[1]=class=Normal name=Normal dimension=1 mean=class=Point name=Unnamed dimension=1 values=[2] sigma=class=Point name=Unnamed dimension=1 values=[4] correlationMatrix=class=CorrelationMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[1]

In [6]:
XRV = ot.RandomVector(X)
Y = ot.CompositeRandomVector(g, XRV)

## Taylor expansion with finite difference gradient

In [7]:
taylor = ot.TaylorExpansionMoments(Y)

In [8]:
taylor.getMeanFirstOrder()[0]

-1.0

In [9]:
taylor.getMeanSecondOrder()[0]

-1.0

In [10]:
sigma2 = taylor.getCovariance()
sigma2

class=CovarianceMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[52]

## If the gradient is known

In [11]:
def myGradient(x):
    dy0dx0 = -x[1]
    dy0dx1 = -x[0]
    gradient = ot.Matrix([[dy0dx0], [dy0dx1]])
    return gradient

In [12]:
xpoint = X.getMean()
myGradient(xpoint).transpose()

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=1 columns=2 values=[-2,-1]

In [13]:
g = ot.PythonFunction(2, 1, mySimulator, gradient=myGradient)

## Taylor decomposition with known gradient

In [14]:
XRV = ot.RandomVector(X)
Y = ot.CompositeRandomVector(g, XRV)

In [15]:
taylor = ot.TaylorExpansionMoments(Y)

In [16]:
taylor.getCovariance()

class=CovarianceMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[52]